# <span style="color:purple">ArcGIS API for Python</span>

<img src="images/Users_02_Admin.PNG" style="width:75%"></img>

## Demo: Administrative Management of Users, Roles & Groups

### We will go over a few workflows to show how the ArcGIS API can help administrators of Web GIS.

Let's import a few modules. We are importing getpass to provide the GIS object with a password in a safe manner.

In [3]:
from arcgis.gis import *
from getpass import getpass
from IPython.display import display

Let's call our GIS object and pass a local Portal.

In [4]:
password = getpass()
gis = GIS("https://martinez10.esri.com/portal/home/", "dmartinez", password, verify_cert=False)

········


In [6]:
gis

GIS @ https://martinez10.esri.com/portal/home/ version:6.1

### Querying for users

Let's run a full query for listed users:

In [7]:
gis.users.search()

[<User username:admin>,
 <User username:dmartinez>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:system_publisher>]

In [8]:
gis.users.me

<User username:dmartinez>

Filter your user search...

In [9]:
gis.users.search("martinez")

[<User username:admin>, <User username:dmartinez>]

Let's search for my colleague, Matt

In [11]:
gis.users.search("admin")

[<User username:admin>]

### Create a new user

Users can be created through the API

In [2]:
gis.users.create?

Object `gis.users.create` not found.


In [13]:
clark = gis.users.create(username = "kal-el", 
                           password="test106xyz3", 
                          firstname="clark", 
                          lastname="kent",
                         description="hello devsummit attendees",
                          email="tester@esri.com",  
                         role='org_publisher')
clark

<User username:kal-el>

This variable is now a "user" object

In [20]:
type(clark)

arcgis.gis.User

Use intellisense to see all the options you have available with user objects.

In [14]:
clark.description

'hello devsummit attendees'

In [15]:
clark.email

'tester@esri.com'

In [16]:
clark.delete()

True

### Querying for groups

In [17]:
gis.groups.search()

[<Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Featured Maps and Apps" owner:admin>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Navigator Maps" owner:esri_nav>]

Let's create the group

In [5]:
gis.groups.create?

In [18]:
# create groups
demo_group = gis.groups.create("PS2019", "Demo; DC")

This is a "group" object

In [19]:
demo_group?

In [18]:
demo_group

<Group title:"DC2019" owner:dmart>

In [20]:
type(demo_group)

arcgis.gis.Group

Like the user object, it has methods we can call on it

In [21]:
demo_group.access

'public'

# Let's say your boss suddenly gets an idea!

<img src="images/idea.jpg" height="600" width="600">

## Boss:
## "I want you to create a new portal, with new users from our regional teams and mulitple groups and make every member is part of their groups. 
## "And I need this before you go home."

<img src="images/worriedsmile.png" height="400" width="400"></img>

## Have no fear! The ArcGIS API for Python is here to help.

We're going to import some libraries to help us

In [22]:
import csv 
import json

# Create Groups

In [23]:
def create_groups():
    print("CREATING GROUPS")

    with open("groups.csv", 'r') as groups_csv:
        groups = csv.DictReader(groups_csv)
        for group in groups:
            try:
                print(" Creating group: ", group['title'], end="  ##  ")
                result = gis.groups.create_from_dict(group)
                if result:
                    print("success")
            except Exception as create_ex:
                print("Error... ", str(create_ex))

# Add Users

In [24]:
def create_users():
    # loop through and create users
    with open("users.csv", 'r') as users_csv:
        users = csv.DictReader(users_csv)
        for user in users:
            try:
                print("Creating user: ", user['username'], end=" ## ")
                result = gis.users.create(username=user['username'],
                                          password=user['password'],
                                          firstname=user['Firstname'],
                                          lastname=user['Lastname'],
                                          email=user['email'],
                                          role =user['role'])
                if result:
                    print("success  ##")

                    print("\t Adding to groups: ", end=" # ")
                    groups = user['groups']
                    group_list = groups.split(",")

                    # Search for the group
                    for g in group_list:
                        group_search = gis.groups.search(g)
                        if len(group_search) > 0:
                            try:
                                group = group_search[0]
                                groups_result = group.add_users([user['username']])
                                if len(groups_result['notAdded']) == 0:
                                    print(g, end =" # ")

                            except Exception as groups_ex:
                                print("\n \t Cannot add user to group ", g, str(groups_ex))
                    print("\n")

            except Exception as add_ex:
                print("Cannot create user: " + user['username'])
                print(str(add_ex))

# Publish Content

In [25]:
def publish_content():
    # Read the csv containing user accounts and their territory info
    csv_path = "users.csv"

    # Read template web map
    template_webmap_dict = dict()
    with open('.\\user_content\\web_map.json', 'r') as webmap_file:
                template_webmap_dict = json.load(webmap_file)

    # Loop through each user and publish the content
    with open(csv_path, 'r') as csv_handle:
        reader = csv.DictReader(csv_handle)
        for row in reader:
            try:
                data_to_publish = '.\\user_content\\' + row['assigned_state'] + ".csv"

                print("Publishing ", data_to_publish, end = " # ")
                added_item = gis.content.add({}, data = data_to_publish)
                published_layer = added_item.publish()

                if published_layer is not None:
                    # publish web map
                    print('webmaps', end= " ## ")
                    user_webmap_dict = template_webmap_dict
                    user_webmap_dict['operationalLayers'][0].update({'itemId': published_layer.itemid,
                                                                     'layerType': "ArcGISFeatureLayer",
                                                                     'title': published_layer.title,
                                                                     'url': published_layer.url + r"/0"})

                    web_map_properties = {'title': '{0} {1} response locations'.format(row['Firstname'], row['Lastname']),
                                          'type': 'Web Map',
                                          'snippet': 'Areas affected by Hurricane Matthew under the supervision of' +\
                                                     '{0} {1}'.format(row['Firstname'], row['Lastname']),
                                          'tags': 'ArcGIS Python API',
                                          'typeKeywords' : "Collector, Explorer Web Map, Web Map, Map, Online Map",
                                          'text': json.dumps(user_webmap_dict)}

                    web_map_item = gis.content.add(web_map_properties)

                    print("success. Assigning to: ", end="  #  ")
                    result1 = published_layer.reassign_to(row['username'])
                    result2 = web_map_item.reassign_to(row['username'])
                    if (result1 and result2) is not None:
                        print(row['username'])
                    else:
                        print("error")
                else:
                    print(" error publishing csv")

            except Exception as pub_ex:
                print("Error : ", str(pub_ex))

# Customize the UX

In [26]:
def set_theme():
    
    folder = r'helper_files'
    gis.admin.ux.set_logo(folder + r'\logo\dc_logo.png')
    gis.admin.ux.set_background(is_built_in=False)
    gis.admin.ux.set_banner(folder + r'\baked_banner\washingtondc_banner.png')
    gis.admin.ux.name = "Washington DC"
    gis.admin.ux.description = "Washington DC"

In [27]:
set_theme()
create_groups()
create_users()
publish_content()

CREATING GROUPS
 Creating group:  Basemaps  ##  success
 Creating group:  Central Services  ##  success
 Creating group:  Compliance  ##  success
 Creating group:  Customer Service, Finance, Billing and Accounting  ##  success
 Creating group:  Demographic Content  ##  success
Creating user:  matthias.schenker ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  jorge.barrachina ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  sean.mcgee ## success  ##
	 Adding to groups:  # Basemaps #  Customer Service # 

Creating user:  mark.jagt ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  gavin.rehkemper ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  rene.rubalcava ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  gerhard.trichtl ## success  ##
	 Adding to groups:  # Basemaps #  Customer Service # 

Creating u

## "We just found out that we have to remove every user from our Portal as we have to use it for another urgent project.


## Good thing we have the ArcGIS API for Python to help.

We're going to run a clean up script(clean.py) in my local IDE(PyCharm) removing everything except the admin account and some groups. 

